In [ ]:
import os
import pickle

import jax
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_disable_jit", True)

import jax.numpy as jnp
import pandas as pd
from jax import random

from metrics.metrics import get_energy_metric

from helpers import apply_function
from losses.loss_and_model import loss_model_bel, loss_model_no_train, loss_model_reparam
from problems.toy_problems import dm_toy_problem, double_well_toy_problem, double_well_toy_problem_opening, ou_toy_problem
from training.train_model import train_model
from problems.diffusion_model import get_problem as get_problem_dm

import datetime
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

from flow_matching_jax.configs.fashion_mnist import get_config
import wandb
import matplotlib.pyplot as plt

print(f"Using Jax Device: {jax.devices()}")

log_dir = "logs/diffusion_model"
writer = tf.summary.create_file_writer(log_dir)

# dim = 1
ts = jnp.linspace(0, 1, 100, dtype=jnp.float32)

# loss_models = [
#     {"function": loss_model_bel, "args": ("first",)},
#     # {"function": loss_model_bel, "args": ("average",)},
#     # {"function": loss_model_bel, "args": ("first",)},
#     # {"function": loss_model_bel, "args": ("last",)},
#     # {"function": loss_model_reparam, "args": ()},
#     # {"function": loss_model_no_train, "args": ()},
# ]

def upper_left_quarter(image):
    h, w = image.shape[:2]
    return image[:h // 2, :w // 2]

def upper_half(image):
    h, w = image.shape[:2]
    return image[:h // 2]

def full_image(image):
    return image

config = get_config()
workdir = "/home/ubuntu/WashingtonMain/conditioning-diffusions/flow_matching_jax/workdir/fashion_mnist"
problem = get_problem_dm(config, workdir, ts, full_image)
# problems = [double_well_toy_problem(3, D=dim), dm_toy_problem(D=dim),  ou_toy_problem(2, D=dim), ou_toy_problem(-2, D=dim), ou_toy_problem(0, D=dim)]
loss_model = loss_model_bel("first")

# jax.config.update('jax_log_compiles', True)

config_template = {
    "seed": 2025,
    "loss_model": "NOT DEFINED",
    "problem": "NOT DEFINED",
    "N_batches": 5000,
    "N_log": 500,
    "N_samples_eval": 16,
    "N_batch_size": 32,
    "ts": ts,
    "n_rngs": 1,
}

rng = random.PRNGKey(config_template["seed"])
rngs = random.split(rng, config_template["n_rngs"])

experiment_dir = f"temp"
os.makedirs(experiment_dir, exist_ok=True)  # Create the directory if it doesn't exist
timestamp = datetime.datetime.now().strftime("%H%M%S-%m%d")
run_dir = f"{log_dir}/run-{timestamp}"
writer = tf.summary.create_file_writer(run_dir)

sde, metrics, get_obs, y_obs, y_init_eval, problem_name = problem
loss_function, nn_model, loss_name = loss_model


from flow_matching_jax.models.conditional_unet import Unet as ConditionalUnet
nn_model = ConditionalUnet(
    dim = 64,
    dim_mults = (1, 2)
)

_t = 0.0
_y = y_init_eval
nn_params = nn_model.init(rng, _t, _y, _y)

wandb.init(project="conditioning-diffusion-models")

final_params, all_metrics, last_metrics = train_model(
                rng,
                ts,
                nn_model,
                nn_params,
                metrics,
                y_obs,
                y_init_eval,
                sde,
                loss_function,
                get_obs,
                N_batches=config_template["N_batches"],
                N_batch_size=config_template["N_batch_size"],
                N_log=config_template["N_log"],
                N_samples_eval=config_template["N_samples_eval"],
            )

wandb.finish()

2025-03-27 08:43:51.320198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743065031.328939   40368 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743065031.333070   40368 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.3.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_min

Using Jax Device: [CudaDevice(id=0)]


I0000 00:00:1743065033.583004   40368 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 94391 MB memory:  -> device: 0, name: NVIDIA GH200 480GB, pci bus id: 0000:dd:00.0, compute capability: 9.0


initializing


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jakiw (jakiwteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Model Training: 100%|██████████| 5000/5000 [37:04<00:00,  2.25it/s]  


grad_variance,█▁▁▁▁▁▁▁▁▁
loss,█▂▂▃▂▂▂▂▁▂
step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
grad_variance,0.4759
loss,1237.4552
step,5000
